## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [4]:
import pandas as pd

In [7]:
! pip install -q kaggle

In [8]:
from google.colab import files

In [9]:
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"santhoshdatasc","key":"7979c55b19f2373206ba6922e0b05ce0"}'}

In [11]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
! cp kaggle.json ~/.kaggle/



In [13]:
! chmod 600 ~/.kaggle/kaggle.json

In [14]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              223KB  2021-04-21 09:40:05           3271  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44           1922  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           1273  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Job Listing 2020                     42MB  2021-03-05 06:59:52            379  
imsparsh/musicnet-dataset                                   MusicNet Dataset

In [16]:
! kaggle competitions download -c fake-news


  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 15.5MB/s]
 53% 5.00M/9.42M [00:00<00:00, 21.9MB/s]
100% 9.42M/9.42M [00:00<00:00, 37.3MB/s]
 46% 17.0M/37.0M [00:00<00:00, 42.0MB/s]
100% 37.0M/37.0M [00:00<00:00, 75.4MB/s]


In [3]:
! kaggle competitions download -c fake-news


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [17]:
! mkdir train



In [19]:
! unzip train.csv.zip -d train



Archive:  train.csv.zip
  inflating: train/train.csv         


In [20]:
df=pd.read_csv('train/train.csv')

In [21]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
###Drop Nan Values
df=df.dropna()


In [23]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [24]:
## Get the Dependent features
y=df['label']

In [25]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [26]:
X.shape

(18285, 4)

In [27]:
y.shape

(18285,)

In [28]:
import tensorflow as tf

In [29]:
tf.__version__

'2.4.1'

In [30]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [31]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [32]:
messages=X.copy()

In [33]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [34]:
messages.reset_index(inplace=True)

In [35]:
import nltk
import re
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [38]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [39]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:10]

[[2321, 688, 1763, 362, 2108, 2861, 2620, 3501, 1513, 1009],
 [4689, 1138, 3212, 4682, 4430, 1918, 1098],
 [1218, 850, 4844, 3307],
 [3473, 1800, 847, 2068, 3547, 4903],
 [1715, 4430, 3928, 3754, 736, 1041, 4430, 4666, 2805, 1045],
 [3270,
  3537,
  855,
  1426,
  2424,
  4312,
  3712,
  690,
  2702,
  2846,
  2786,
  1211,
  237,
  4690,
  1098],
 [4530, 1704, 74, 4906, 3715, 4415, 409, 564, 4640, 1400, 2310],
 [909, 1171, 4147, 2438, 2313, 3509, 4312, 6, 4640, 1400, 2310],
 [1381, 4186, 1854, 4640, 2532, 4879, 2124, 448, 4312, 70],
 [3241, 3245, 3221, 2217, 1297, 3578, 1826, 2400]]

### Embedding Representation

In [40]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3501 1513 1009]
 [   0    0    0 ... 4430 1918 1098]
 [   0    0    0 ...  850 4844 3307]
 ...
 [   0    0    0 ... 4640 1400 2310]
 [   0    0    0 ...  828 1460  776]
 [   0    0    0 ... 3048 3369 2040]]


In [41]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2321,
        688, 1763,  362, 2108, 2861, 2620, 3501, 1513, 1009], dtype=int32)

In [42]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
len(embedded_docs),y.shape

(18285, (18285,))

In [45]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [46]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [48]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 54ms/step - loss: 0.4596 - accuracy: 0.7582 - val_loss: 0.2056 - val_accuracy: 0.9120
Epoch 2/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1465 - accuracy: 0.9457 - val_loss: 0.2043 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0984 - accuracy: 0.9684 - val_loss: 0.2362 - val_accuracy: 0.9132
Epoch 4/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0685 - accuracy: 0.9781 - val_loss: 0.2733 - val_accuracy: 0.9105
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0405 - accuracy: 0.9888 - val_loss: 0.3012 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0293 - accuracy: 0.9915 - val_loss: 0.4024 - val_accuracy: 0.9075
Epoch 7/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0247 - accuracy: 0.9926 - val_loss: 0.4358 - val_accuracy: 0.910

### Performance Metrics And Accuracy

In [49]:

y_pred1=model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred1)

array([[3076,  343],
       [ 212, 2404]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.908036454018227

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

